# Using BFGP++ in the Unified Planning library

In this notebook, we show the steps to take to compute generalized solutions for planning problems with the _BFGP++_ UP interface. These can be obtained from the combination of three different execution modes and several target languages, to address STRIPS, program synthesis and learning action models benchmarks.   

## 1. Installation

In [ ]:
!pip install up-bfgp


## 2. Execution modes

The framework allows three different kinds of execution modes, `synthesis`, `validation` and `repair`. The next steps describes each mode and how to use them in a particular example for solving _Gripper_ domain.

First let's import the required libraries:

In [ ]:
from unified_planning.shortcuts import *  # type: ignore
from unified_planning.plans import PlanKind  # type: ignore
from unified_planning.engines.results import PlanGenerationResultStatus

The execution of the framework requires a `domain_file` name, a list of `problem_files` names and the set of arguments (`args`) which depend on the executed mode. The following code serve as a base for calling _BFGP++_:

In [ ]:
def base_bfgp_call(domain_file: str, problem_files: List[str], args: dict):
    with up.environment.get_environment().factory.FewshotPlanner(name='bfgp') as bfgp:
        bfgp.set_arguments(**args)
        bfgp_problems = bfgp.generate_problems(domain_file, problem_files)
        # Compute the generalized plan for these input problems
        result = bfgp.solve(bfgp_problems, output_stream=sys.stdout)
        # Check whether all generated plans are satisficing
        return all(r == PlanGenerationResultStatus.SOLVED_SATISFICING for r in result)

This function assumes the domain and problems are existing files, however, they can also be implemented in Python code as shown in many other usage examples in the Unified Planning library. If the list of `problems` is implemented in Python instead of parsed from a file, _BFGP++_ can be directly called with the `bfgp.solve(problems)` command (i.e., skipping `bfgp.generate_problems(domain_file, problem_files)`) . 



### 2.a. Synthesis

This mode refers to the generation of a planning program given a domain and a set of planning instances in that domain. For instance, the _Gripper_ domain is defined by 3 sorts of objects (_room_, _ball_, and _gripper_); 4 predicates that refer to whether the robby is (_robby-at_), whether each ball is in a specific room (_at_), if a gripper is free (_free_) or if it is carrying a ball (_carry_); 2 sorts of constants which refer to _left_ and _right_ grippers, and rooms _A_ and _B_; and 3 action schemas that describe the picking and dropping of a ball with a gripper in a room, and an action for moving from one room to another one. Then, each _Gripper_ instance is initialized with the robby and all balls in room _A_ and the goal is to move all balls to room _B_. A general strategy to solve any problem consists of picking a ball, move to room _B_, drop it, move to room _A_, and repeat this procedure while there are still balls in room _A_. 

In [ ]:
"""BFGP++ synthesizing a program that can solve 3 different Gripper instances"""
domain = '../tests/domains/gripper/domain.pddl'
problems = [f'../tests/domains/gripper/p{i:02}.pddl' for i in range(1, 4)]
kwargs = dict({'mode': 'synthesis',
               'theory': 'cpp',
               'program_lines': 10,
               'program': 'gripper',
               'translated_problem_dir': 'tmp_gripper/'})

print(f"Does a solution exist for Gripper? {base_bfgp_call(domain_file=domain, problem_files=problems, args=kwargs)}")


### 2.b. Validation

In the `synthesis` mode example above, the Gripper solution is first found, then generates one plan per instance and validates those plans with the UP library. Now let's assume that we have the program, e.g. the Gripper solution, and we want to validate it over new planning instances, e.g. starting with 10 and 13 balls in room A, so we run the following command to validate the program:

In [ ]:
"""BFGP++ validating a program over 2 different and larger Gripper instances"""
domain = '../tests/domains/gripper/domain.pddl'
problems = [f'../tests/domains/gripper/p{i:02}.pddl' for i in range(4, 6)]
kwargs = dict({'mode': 'validation-prog',
               'theory': 'cpp',
               'program': 'gripper',
               'translated_problem_dir': 'tmp_gripper/'})
print(f"Is the program valid over new Gripper instances? {base_bfgp_call(domain_file=domain, problem_files=problems, args=kwargs)}")

 

### 2.c. Repair

The two modes above can be interpreted as if the user has no knowledge at all about the solution (empty program, hence run `synthesis`) or full knowledge about it (full program is known, hence run `validation`). Thus, `repair` mode is an intermediate step between `synthesis` and `validation`, where some parts of the program may be known (also they are maybe wrong) and the system must repair them to get to a valid solution if one exists.  The following examples try to repair 3 different input programs where the first is all correct, in the second all loops are missing, and in the third all planning actions are missing:

In [ ]:
"""BFGP++ repairing (if needed) 3 Gripper programs for the same input instances"""
domain = '../tests/domains/gripper/domain.pddl'
problems = [f'../tests/domains/gripper/p{i:02}.pddl' for i in range(1, 4)]

# Example 1: the input program is correct
kwargs = dict({'mode': 'repair',
               'theory': 'cpp',
               'program_lines': 10,
               'program': '../tests/domains/gripper/program_repair/gripper_cpp_ok',
               'translated_problem_dir': 'tmp_gripper_ok/'})

print(f"Example 1 has been repaired? {base_bfgp_call(domain_file=domain, problem_files=problems, args=kwargs)}")

# Example 2: the input program has some missing loops
kwargs = dict({'mode': 'repair',
               'theory': 'cpp',
               'program_lines': 10,
               'program': '../tests/domains/gripper/program_repair/gripper_cpp_missing_loops',
               'translated_problem_dir': 'tmp_gripper_missing_loops/'})

print(f"Example 2 has been repaired? {base_bfgp_call(domain_file=domain, problem_files=problems, args=kwargs)}")

# Example 3: the input program has some missing planning action
kwargs = dict({'mode': 'repair',
               'theory': 'cpp',
               'program_lines': 10,
               'program': '../tests/domains/gripper/program_repair/gripper_cpp_missing_planning_actions',
               'translated_problem_dir': 'tmp_gripper_missing_planning_actions/'})

print(f"Example 3 has been repaired? {base_bfgp_call(domain_file=domain, problem_files=problems, args=kwargs)}")

## 3. Target Languages
There are two major sorts of target languages in this system:
1. the ones used for computing generalized planning solutions, and 
2. the ones to learn action models.
 
In the former, we set the `theory` argument either to `assembler` (looping and branching are goto instructions) or `cpp` (looping and branching are well-structured and programs are a restricted set of C++). In the latter, we set the `theory` argument to one of the four different languages for representing deterministic transitions, `actions_strips` for STRIPS action schemas, `actions_adl` for action schemas with quantified effects, `actions_cell` for representing 1D cellular automata transitions, and `actions_ram` for action schemas with loops and branches.

### 3.a. STRIPS benchmarks
These are the set of problems represented in STRIPS language plus negative preconditions. All the executions of the _Gripper_ domain above are of this kind, with the `cpp` target language. Now let's try to compute an assembly-like program for the same domain (note the setting update `'theory': 'assembler'`)


In [ ]:
"""BFGP++ synthesizing an assembler-like program that can solve 3 different Gripper instances"""
domain = '../tests/domains/gripper/domain.pddl'
problems = [f'../tests/domains/gripper/p{i:02}.pddl' for i in range(1, 4)]
kwargs = dict({'mode': 'synthesis',
               'theory': 'assembler',
               'program_lines': 8,
               'program': 'gripper',
               'translated_problem_dir': 'tmp_gripper_assembler/',
               'evaluation_functions': ["ed", "lc"]})

print(f"Does an assembly-like solution exist for Gripper? {base_bfgp_call(domain_file=domain, problem_files=problems, args=kwargs)}")



### 3.b. Program Synthesis
These benchmarks extend STRIPS and negative preconditions with numerical state variables (i.e., integers) and operations over them like increase, decrease, add, subtract, smaller, greater and so on. We split them into two classes, the first focus on computing the n-th term of some sequence, e.g. _Triangular Sum_, and the second consists of list manipulation, e.g. _Sorting_. The following code exemplifies a call for _Sorting_ domain that arranges any list of numbers in ascending order: 

In [ ]:
"""BFGP++ synthesizing a Sorting algorithm given 10 instances with an arbitrary lists of numbers"""
domain = '../tests/domains/sorting/domain.txt'
problems = [f'../tests/domains/sorting/{i}.txt' for i in range(1, 11)]
kwargs = dict({'mode': 'synthesis',
               'theory': 'cpp',
               'program_lines': 8,
               'program': 'sorting',
               'translated_problem_dir': 'tmp_sorting/'})

print(f"Does a sorting algorithm of 8 lines exist? {base_bfgp_call(domain_file=domain, problem_files=problems, args=kwargs)}")


### 3.c. Learning Action Models
These benchmarks are similar to STRIPS with negative preconditions, but every instance represents a deterministic transition (pre-state is the initial state, and post-state is the goal), hence they need to include information about the executed action as a predicate, e.g. for picking a ball in room _A_ in the _Gripper_ domain, the following fact `(action_pick ball1 rooma right)` must appear in the initial state. The following shows an example for learning the picking action of _Gripper_ given 20 picking deterministic transitions.



In [ ]:
"""BFGP++ learning a STRIPS action model for picking balls in Gripper from the first 20 instances"""
domain = '../up_bfgp/bfgp_pp/domains/aiplan4eu/action_models/synthesis/strips/gripper/pick/domain.txt'
problems = [f'../up_bfgp/bfgp_pp/domains/aiplan4eu/action_models/synthesis/strips/gripper/pick/{i}.txt' for i in range(1, 21)]
kwargs = dict({'mode': 'synthesis',
               'theory': 'actions_strips',
               'program': 'gripper_pick',
               'evaluation_functions': ["ilc", "mi", "cwed"],
               'translated_problem_dir': 'tmp_gripper_pick/'})

print(f"Does the pick action schema in Gripper be learned? {base_bfgp_call(domain_file=domain, problem_files=problems, args=kwargs)}")

From the execution above in the action STRIPS theory, only 58 out of 82 lines are required, and can be summarized as apply the effects (lines 54-56) if the preconditions (lines 0-53) hold in the current state. For more details about this target languages, we refer to our [ECAI'23 article](https://ebooks.iospress.nl/doi/10.3233/FAIA230502).

## 4. Input Arguments

Beyond the execution mode and target language, there are other input arguments which might be used to finetune the searching mechanism or structure the outputs. Here there is a list:
- `program_lines` (integer): number of program lines
- `program` (string): where to read or write a program file
- `translated_problem_dir` (string): folder to write all output files
- `evaluation_functions` (list of strings): list of evaluation functions, where nodes in the search are prioritized by smaller values on the left and breaking ties with the values on the right. Here we show some of the available evaluation functions:
    * `ed`: Euclidean distance from the current state to a goal condition (aggregated over all input instances)
    * `lc`: loop counter (number of goto or for instructions)
    * `nei`: number of empty instructions in a program
    * `mri`: the maximum among each number of instruction occurrences in a program
    * `ilc`: negated loop counter (prioritizes programs with more loops)
    * ...    
- `num_extra_pointers` (integer): the number of pointers is computed by the maximum number of occurrences of each type in the action and predicate parameters, so they are increased by this value which is 0 by default. 

The following is an example that use all the parameters above:

In [ ]:
"""Testing the BFGP++ solver can solve 50 first Miconic instances"""
domain = '../tests/domains/miconic/domain.pddl'
problems = [f'../tests/domains/miconic/training/easy/p{i:02}.pddl' for i in range(1, 51)]
kwargs = dict({'mode': 'synthesis',
               'theory': 'cpp',
               'program_lines': 13,
               'program': 'miconic',
               'num_extra_pointers': 1,
               'evaluation_functions': ["ilc", "ed"],
               'translated_problem_dir': 'tmp_miconic/'})

print(f"Does a solution exist for Miconic? {base_bfgp_call(domain_file=domain, problem_files=problems, args=kwargs)}")